In [ ]:
# !uv pip install geemap
# !pip install geopandas
# !pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 201.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 232.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 192.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [geopandas]/4 [geopandas]


# Water Quality Parameter Viz
### A simple test site visualization for outside siliguri

The idea of this simple colab notebook is to showcase the one of many water quality parameters through the Sentinal-2 Surface Reflectance Imagery.

#### The reason of Surface Reflectance
The reason behindthe surface reflectance data is used because it is already  atmospheric deviations corrected.



# Implementing ideas for this study

## A. Securing a study area
For this the river, situated in the outskirt of city Siliguri Mahananda River is chosen.

## B. Clipping the Sentinal 2 imagery
Clipping is needed for the satellite imagery extended to the study area.

### C. Methodology
The following is the tentative methodology for this small study.

1. **Acquiring seasonal *blended* imagery**. The seasonal range will only be for the year 2024 and are as follows -
    *   **Winter** - December, January, February.
    *   **Pre Monsoon** - March, April, May.
    *   **Monsoon** - June , July, August.
    *   **Post Monsoon** - September, October, November
2. **Water body separation**. Using Normalized Difference Water Index (NDWI). We will seprate the water body as a single-band image where each pixel value represents the calculated NDWI for that location. The pixel values will typically range from -1 to +1, with higher positive values indicating the presence of water.*
3. Further analysis


"* -This is because for WQI I am not sure if the NDWI is needed after all!"

# A. From KMZ to shp
This part will deal with the conversion to shp for the better processing by google earth engine.

The tasks are -
1. Authenticate with GEE
2. Upload the file
3. Convert the file

In [ ]:
# Library import section
import geemap # Geemap usage
import ee # Earth Engine Authentication - Important library
import geopandas as gpd
import zipfile
import os

In [ ]:
try:
    ee.Authenticate()
    ee.Initialize(project="ee-workspacesohan")
    print('Google Earth Engine has been successfully initialized.')
except Exception as e:
    print('Google Earth Engine authentication failed:', e)
    print('Please authenticate to your Google account by running the following command:')
    print('ee.Authenticate()')

Google Earth Engine has been successfully initialized.


Convertion of kmz to shp

In [ ]:
# Run only and only to convert kmz/kml to shp if needed!
"""
def kmz_to_shp(input_kmz, output_shp):
    """
    # Converts a KMZ file to an ESRI Shapefile.

    # Args:
        # input_kmz (str): The path to the input KMZ file.
        # output_shp (str): The desired path for the output Shapefile.
    """
    # Create a temporary directory to extract the KML file
    temp_dir = 'temp_kml_extraction'
    os.makedirs(temp_dir, exist_ok=True)

    try:
        # Open the KMZ file as a zip archive
        with zipfile.ZipFile(input_kmz, 'r') as zip_ref:
            # Extract all contents to the temporary directory
            zip_ref.extractall(temp_dir)

        # Find the KML file within the extracted contents (usually doc.kml)
        kml_file = None
        for root, _, files in os.walk(temp_dir):
            for file in files:
                if file.endswith('.kml'):
                    kml_file = os.path.join(root, file)
                    break
            if kml_file:
                break

        if not kml_file:
            print(f"Error: No KML file found inside {input_kmz}")
            return

        # Read the KML file using geopandas
        print(f"Reading data from {kml_file}...")
        gdf = gpd.read_file(kml_file, driver='KML')

        # Write the GeoDataFrame to a shapefile
        print(f"Writing data to {output_shp}...")
        gdf.to_file(output_shp, driver='ESRI Shapefile')

        print("Conversion successful!")

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Clean up the temporary directory
        if os.path.exists(temp_dir):
            for root, dirs, files in os.walk(temp_dir, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(temp_dir)


# Example usage:
if __name__ == '__main__':
    # Replace with your file paths
    input_kmz_file = '/content/drive/MyDrive/Personal Project and Workspace/Projects/SiDCREW Project/Test Study Area Water Quality/Test Site - Siliguri 1.kmz'
    output_shp_file = '/content/drive/MyDrive/Personal Project and Workspace/Projects/SiDCREW Project/Test Study Area Water Quality/Test Site - Siliguri 1.shp' # Remember not to include any space in the output shapefile naming convention

    kmz_to_shp(input_kmz_file, output_shp_file)


## Load featurecollection

### Subtask:
Load the shapefile asset from Google Earth Engine using its asset ID.


**Reasoning**:
Define the asset ID and load the shapefile asset from Google Earth Engine.



In [ ]:
asset_id = 'projects/ee-workspacesohan/assets/Test_Site-Siliguri_1'
study_area = ee.FeatureCollection(asset_id)

In [ ]:
# Visulazing the map

# Initialize a map
Map = geemap.Map(center=[26.65, 88.41], zoom=13) # You might want to adjust the center and zoom level
Map.add_basemap("Esri.WorldImagery")

# Add the study area FeatureCollection to the map
style = {'color': 'ff0000ff', 'width': 3, 'lineType': 'dashed', 'fillColor': '00000000'}
Map.add_layer(study_area, style, 'Study Area')

# Display the map
Map

Map(center=[26.65, 88.41], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

# B. Clipping the Sentinal 2 SR (Surface Reflectance) images
*  For this task we will first try to segregate the images of Sentinal 2 SR into three distinct seasonal variables.
*  We will clip those into the study area

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11 # high-altitude clouds made of ice crystals - can interfere with the signals received by the sensor and affect the accuracy of the data

  # Both flags should be set to zero, indicating clear conditions
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset_summer = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2024-03-01', '2024-05-31')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 9))
    .map(mask_s2_clouds).filterBounds(study_area)
)
print(dataset_summer.size().getInfo())

visualizationNIR = {
    'bands': ['B8', 'B4', 'B3'],
    'gamma': 0.9,
    'min': 0.08152200169861316,
    'max': 0.30841801054775714
}

# visualizationRGB = {
#     # 'min': 0.05265999957919121,
#     # 'max': 0.3667599856853485,
#     'bands': ['B4', 'B3', 'B2'],
#     # 'gamma': 0.9
# }

Map = geemap.Map()
Map.add_basemap("Esri.WorldImagery")
Map.set_center(88.41, 26.65, 13)
# Clip the mean composite to the study area before adding to the map
Map.add_layer(dataset_summer.mean().clip(study_area), visualizationNIR, 'NIR (Clipped)')
# Map.add_layer(dataset_summer.mean().clip(study_area), visualizationRGB, 'RGB (Clipped)')
Map